# 1. Split the data into folds for training/validation/test

In [ ]:
import os
import sys

# Add scripts folder to path
sys.path.append(os.path.join(os.pardir, 'scripts'))

from configure_nifti_folds import configure_nifti_folds

data_folder = os.path.join(os.path.expanduser('~'), 'data', 'IXI', 'resized')
project_folder = os.path.join(os.path.expanduser('~'), 'projects', 'brain-age')

configure_nifti_folds(folders=[data_folder], targets='age', stratification=['age', 'sex', 'scanner'],
                      k=5, test_portion=0.2, destination=os.path.join(project_folder, 'data'))

# 2. Configure an SFCN regression model

In [ ]:
import json

from configure_model import configure_model

model_kwargs = {
    'input_shape': [43, 54, 41],
    'dropout': 0.2,
    'weight_decay': 1e-3,
    'prediction_range': [19, 87]
    
}
model_kwargs = json.dumps(model_kwargs)

configure_model(model='sfcn-reg', kwargs=model_kwargs, destination=os.path.join(project_folder, 'model'))

# 3. Configure a preprocessor, an augmenter and a learning rate schedule

In [ ]:
from pyment.data.augmenters import NiftiAugmenter
from pyment.data.preprocessors import NiftiPreprocessor
from pyment.utils.learning_rate import LearningRateSchedule

# Create a preprocessor which normalizes the images to the range [0, 1]
preprocessor = NiftiPreprocessor(sigma=255.)
preprocessor.save(os.path.join(project_folder, 'preprocessor.json'))
print(preprocessor)

augmenter = NiftiAugmenter(flip_probabilities=[0.5, 0, 0])
augmenter.save(os.path.join(project_folder, 'augmenter.json'))
print(augmenter)

learning_rate_schedule = LearningRateSchedule({0: 1e-3, 20: 3e-3, 40: 1e-4, 60: 3e-4})
learning_rate_schedule.save(os.path.join(project_folder, 'learning_rate_schedule.json'))
print(learning_rate_schedule)

In [ ]:
from shutil import rmtree

from fit_model import fit_model

run_folder = os.path.join(project_folder, 'run')

if os.path.isdir(run_folder):
    rmtree(run_folder)
    
fit_model(model=os.path.join(project_folder, 'model'),
          training=[os.path.join(project_folder, 'data', f'fold_{i}.json') \
                    for i in range(4)],
          validation=[os.path.join(project_folder, 'data', f'fold_4.json')],
          preprocessor=os.path.join(project_folder, 'preprocessor.json'),
          augmenter=os.path.join(project_folder, 'augmenter.json'),
          batch_size=4,
          num_threads=8,
          loss='mse',
          metrics=['mae'],
          learning_rate_schedule=os.path.join(project_folder, 'learning_rate_schedule.json'),
          epochs=5,
          domain=None,
          destination=run_folder)